In [5]:
import numpy as np
import itertools
import random
from random import getrandbits, randint
from functools import reduce
import time
import PIL.Image, PIL.ImageDraw
import pandas as pd

In [6]:
_best_ugl = [
                            " ", #0
                            "╴", #1
                            "╷", #2
                            "┐", #3
                            "╶", #4
                            "─", #5
                            "┌", #6
                            "┬", #7
                            "╵", #8
                            "┘", #9
                            "│", #10
                            "┤", #11
                            "└", #12
                            "┴", #13
                            "├", #14
                            "┼", #15
                        ]
def valid_co(x, y, n, m):
    return (x>=0) and (x<n) and (y>=0) and (y<m)
def show_nice(a):
    for id, i in enumerate(a):
        for ud, u in enumerate(i):
            print(" ", end="")
            if u[0]:
                print("─"*3, end="")
            else:
                print(" "*3, end="")
        print(" ")
        for ud, u in enumerate(i):
            if u[3]:
                print("│", end="")
            else:
                print(" ", end="")
            print(" "*3, end="")
        print("│")
    for ud, u in enumerate(i):
        print(" ", end="")
        print("─"*3, end="")
    print(" ")

In [108]:
def Kruskal(n, m):
    a = np.ones((n,m,4), dtype=np.int32)
    b = np.zeros((n,m), dtype=np.int32)
    def get_way(px,py):
        if px == 0:
            if py == 1:
                return 1
            else:
                return 3
        else:
            if px == 1:
                return 2
            else: return 0
    for i in range(n):
        for u in range(m):
            b[i,u] = i*m + u
    wall = np.array([(i,u,j) for i in range(n) for u in range(m) for j in (1,2) if ((i<n-1) or (j==1)) and ((u<m-1) or (j==2))], dtype=np.int32)
    np.random.shuffle(wall)
    for ix,iy,ij in wall:
        #print(ix, iy, ij)
        #show_nice(a)
        #print(a)
        #print(b)
        #print()
        if ij == 1:
            if b[ix, iy] != b[ix, iy+1]:
                a[ix, iy, 1] = 0
                a[ix, iy+1, 3] = 0
                if b[ix, iy] < b[ix, iy+1]:
                    link = b[ix, iy]
                    stack = [(ix, iy+1)]
                else:
                    link = b[ix, iy+1]
                    stack = [(ix, iy)]
            else:
                continue
        else:
            if b[ix, iy] != b[ix+1, iy]:
                a[ix, iy, 2] = 0
                a[ix+1, iy, 0] = 0
                if b[ix, iy] < b[ix+1, iy]:
                    link = b[ix, iy]
                    stack = [(ix+1, iy)]
                else:
                    link = b[ix+1, iy]
                    stack = [(ix, iy)]
            else:
                continue
        while stack:
            x, y = stack.pop()
            
            #print("stack=>",x,y)
            if b[x, y] == link:
                continue
            else:
                b[x, y] = link
            for px, py in ((-1, 0), (1, 0), (0, 1), (0, -1)):
                if valid_co(x+px,y+py,n,m) and (a[x,y,get_way(px,py)] == 0) and (b[x+px,y+py] != link):
                    stack.append((x+px,y+py))
    #print(b)
    return a

In [109]:
def count_no_way(a):
    nway = 0
    for i in a:
        for u in i:
            if sum(u) == 3:
                nway += 1
    return nway

In [196]:
sr = 0
for i in range(1000):
    a = Kruskal(10,10)
    sr += count_no_way(a)
print(sr, sr/1000/1)

30358 30.358


In [189]:
def B2_3(n, m):
    a = np.zeros((n+1, m+1, 4), dtype=np.int32)
    b = np.zeros((n+1, m+1), dtype=np.int32)
    for i in range(1, n):
        for u in range(1, m):
            b[i,u] = i*m + u
    p = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    def _k(k):
        return (k+2)%4
    for i in range(1, n):
        for u in range(1, m):
            good = [k for k in range(4) if b[i,u] != b[i+p[k][0],u+p[k][1]]]
            k = random.choice(good)
            ii, uu = i+p[k][0], u+p[k][1]
            a[i,u,k] = 1
            a[ii,uu,_k(k)] = 1
            #print(k, i,u,ii,uu)
            #print(a)
            #print(b)
            #print()
            if b[i,u]<b[ii,uu]:
                link = b[i,u]
                stack = [(ii,uu)]
            else:
                link = b[ii,uu]
                stack = [(i,u)]
            while stack:
                x, y = stack.pop()
                #print('stack =>', x, y)
                if b[x, y] == link:
                    continue
                else:
                    b[x, y] = link
                for k in range(4):
                    if a[x,y,k] and (b[x+p[k][0],y+p[k][1]]!=link):
                        stack.append((x+p[k][0], y+p[k][1]))
    #print(a)
    #print(b)
    return a

In [190]:
def wall_to_norm(a):
    #print("SHAPR", a.shape)
    d = np.ones((a.shape[0]-1,a.shape[1]-1,4), dtype=np.bool)
    for i in range(1, a.shape[0]):
        for u in range(1, a.shape[1]):
            if (u!=a.shape[1]-1) and (not a[i, u, 0]):
                d[i-1, u-1, 1] = False
                d[i-1, u, 3] = False
            if (i!=a.shape[0]-1) and (not a[i, u, 3]):
                d[i-1, u-1, 2] = False
                d[i, u-1, 0] = False
    return d

In [192]:
a = B2_3(50,25)
d = wall_to_norm(a)
show_nice(d)

 ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── 
│   │           │               │   │       │           │           │       │   │           │       │
         ───         ─── ───         ───     ─── ───                 ───             ─── ─── ───     
│       │   │           │               │                   │   │   │           │           │   │   │
                 ───     ───     ───             ─── ─── ───     ───     ───         ─── ───         
│   │       │   │       │   │       │       │               │   │   │       │   │   │       │       │
 ───     ─── ───     ───     ─── ───     ─── ───             ───     ───     ───         ─── ───     
│           │   │           │                   │   │   │       │                                   │
 ───     ───         ─── ─── ─── ───         ───         ───         ─── ─── ─── ───     ───     ─── 
│       │                   │       │   │   │   │   │       │   │               │ 

     ─── ───     ───         ─── ─── ───     ─── ───     ─── ─── ─── ───     ───     ─── ───         
│   │           │       │   │       │       │       │           │       │               │       │   │
 ───     ─── ─── ───                 ───     ───     ─── ───     ───     ───         ───     ───     
│           │   │   │       │   │                       │       │           │   │       │   │       │
 ───                 ─── ─── ───     ─── ───         ─── ───     ───     ───         ───             
│       │   │   │           │       │       │   │           │                   │       │       │   │
 ───             ───                 ───     ───     ─── ─── ───     ─── ─── ───         ─── ─── ─── 
│   │   │               │   │   │       │       │   │                   │           │       │       │
     ───             ───         ─── ─── ───         ─── ───         ───     ───     ───             
│       │   │   │   │       │           │       │   │       │   │   │   │   │     

In [195]:
sr = 0
for i in range(1000):
    a = wall_to_norm(B2_3(10,10))
    sr += count_no_way(a)
print(sr, sr/1000/1)

28842 28.842


In [199]:
def _rr_way(a, x, y, xx, yy, z):
    if (x == a.shape[0]-1) and (y == a.shape[1]-1):
        return z
    if (not a[x,y,1]) and (y+1 != yy):
        mb = _rr_way(a, x, y+1, x, y, z+1)
        if mb is not None:
            return mb
    if (not a[x,y,2]) and (x+1 != xx):
        mb = _rr_way(a, x+1, y, x, y, z+1)
        if mb is not None:
            return mb
    if (not a[x,y,0]) and (x-1 != xx):
        mb = _rr_way(a, x-1, y, x, y, z+1)
        if mb is not None:
            return mb
    if (not a[x,y,3]) and (y-1 != yy):
        mb = _rr_way(a, x, y-1, x, y, z+1)
        if mb is not None:
            return mb
def rr_way(a):
    return _rr_way(a, 0, 0, 0, 0, 0)

In [213]:
sr = 0
for i in range(1000):
    a = wall_to_norm(B2_3(10,10))
    sr += rr_way(a)
print(sr, sr/100/10)

22692 22.692


In [214]:
sr = 0
for i in range(1000):
    a = Kruskal(10,10)
    sr += rr_way(a)
print(sr, sr/100/10)

23338 23.338


In [41]:
for i in itertools.permutations((-1, 1,), 2):
    print(i)

(-1, 1)
(1, -1)


In [249]:
def B2_4(n, m):
    a = np.zeros((n+1, m+1, 4), dtype=np.bool)
    b = np.zeros((n+1, m+1), dtype=np.int32)
    b[:,0] = 1
    b[:,-1] = 1
    b[0,:] = 1
    b[-1,:] = 1
    lost = (n-1)*(m-1)
    now = 1
    p = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    def _k(k):
        return (k+2)%4
    while lost:
        now += 1
        x, y = randint(1, n-1), randint(1, m-1)
        while (b[x, y]):
            x, y = randint(1, n-1), randint(1, m-1)
        while True:
            #print(x, y)
            #show_nice(wall_to_norm(a))
            #print(b)
            if b[x, y]!=now:
                b[x, y] = now
                lost -= 1
            good = [k for k in range(4) if b[x+p[k][0], y+p[k][1]] != now]
            if not good:
                x, y = randint(1, n-1), randint(1, m-1)
                while (b[x, y]) != now:
                    x, y = randint(1, n-1), randint(1, m-1)
                continue
            k = random.choice(good)
            xx, yy = x+p[k][0], y+p[k][1]
            #print("  ->'",xx,yy,b[xx,yy])
            a[x,y,k] = True
            a[xx,yy,_k(k)] = True
            if b[xx,yy] !=0:
                break
            x, y = xx, yy
    return a

In [251]:
a = B2_4(100, 25)
d = wall_to_norm(a)
show_nice(d)

 ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── 
│   │                       │           │                               │                   │       │
         ───     ─── ─── ───     ───     ─── ─── ───     ───     ───     ───         ───     ───     
│   │   │   │       │   │   │   │               │       │           │   │       │       │           │
                 ───                 ─── ───         ─── ─── ─── ───     ───     ─── ─── ───         
│       │   │   │       │       │       │       │   │   │       │               │           │   │   │
     ───                             ─── ───                 ─── ───     ─── ───     ───         ─── 
│       │   │   │   │       │   │   │               │   │           │   │           │   │           │
             ─── ───     ───     ───     ─── ───         ───         ─── ───     ───         ─── ─── 
│   │                   │   │       │   │               │   │   │           │     

│   │   │   │   │       │   │       │   │   │           │   │   │   │       │   │           │   │   │
                 ─── ───         ───     ─── ─── ───     ───     ─── ───             ───             
│   │   │                                       │   │               │       │   │   │   │   │       │
         ─── ───             ─── ─── ───     ───         ─── ───     ─── ───             ───     ─── 
│               │   │   │   │       │       │           │                                   │   │   │
     ───     ───     ─── ───     ───             ───             ───         ───     ───             
│   │   │   │                   │       │       │   │   │   │   │   │   │   │       │               │
 ───         ─── ───         ───     ─── ─── ───     ─── ─── ───         ───     ─── ─── ─── ───     
│       │       │       │   │           │   │                       │       │   │   │   │   │       │
 ───         ───     ─── ─── ─── ─── ───     ───     ─── ─── ───     ─── ───      

│               │       │   │   │   │   │           │   │       │   │           │   │       │       │
     ─── ───     ───                 ───     ───     ─── ───     ───     ───                 ───     
│       │                       │           │   │   │   │           │   │           │   │       │   │
     ─── ─── ─── ───                     ───                 ─── ───     ─── ─── ───     ─── ─── ─── 
│   │   │           │   │   │       │   │       │   │       │       │           │   │   │   │       │
                 ───     ───         ───                 ─── ───     ───     ───     ───     ───     
│   │   │   │   │       │   │   │       │   │   │   │           │           │                   │   │
         ───         ───         ─── ─── ───     ───     ─── ───     ─── ───     ───     ─── ───     
│   │       │   │   │                                   │   │               │       │           │   │
 ─── ───             ───     ───             ───     ───                 ─── ───  

In [254]:
sr = 0
for i in range(100):
    a = wall_to_norm(B2_4(100,100))
    sr += count_no_way(a)
print(sr, sr/100/100)

277442 27.7442


In [261]:
sr = 0
for i in range(100):
    a = wall_to_norm(B2_4(100,100))
    sr += rr_way(a)
print(sr, sr/100/100)

44360 4.436


In [256]:
def B2_5(n, m):
    a = np.zeros((n+1, m+1, 4), dtype=np.bool)
    b = np.zeros((n+1, m+1), dtype=np.int32)
    b[:,0] = 1
    b[:,-1] = 1
    b[0,:] = 1
    b[-1,:] = 1
    lost = (n-1)*(m-1)
    now = 1
    p = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    def _k(k):
        return (k+2)%4
    while lost:
        now += 1
        x, y = randint(1, n-1), randint(1, m-1)
        while (b[x, y]):
            x, y = randint(1, n-1), randint(1, m-1)
        while True:
            #print(x, y)
            #show_nice(wall_to_norm(a))
            #print(b)
            if b[x, y]!=now:
                b[x, y] = now
                lost -= 1
            good = [k for k in range(4) if b[x+p[k][0], y+p[k][1]] == 0]
            if not good:
                good = [k for k in range(4) if b[x+p[k][0], y+p[k][1]] != now]
            if not good:
                x, y = randint(1, n-1), randint(1, m-1)
                while (b[x, y]) != now:
                    x, y = randint(1, n-1), randint(1, m-1)
                continue
            k = random.choice(good)
            xx, yy = x+p[k][0], y+p[k][1]
            #print("  ->'",xx,yy,b[xx,yy])
            a[x,y,k] = True
            a[xx,yy,_k(k)] = True
            if b[xx,yy] !=0:
                break
            x, y = xx, yy
    return a

In [258]:
a = B2_4(100, 25)
d = wall_to_norm(a)
show_nice(d)

 ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── 
│                                   │       │                   │   │   │       │                   │
     ───             ───     ───         ───                 ───             ─── ───         ─── ─── 
│   │       │   │   │       │   │           │   │   │   │           │               │   │           │
     ─── ───                     ───             ─── ─── ───     ─── ───     ─── ───     ─── ───     
│   │   │   │   │   │   │   │   │   │   │               │                           │   │   │       │
 ───         ─── ───     ───         ───     ─── ───     ───     ───     ─── ─── ───         ─── ─── 
│                       │   │           │       │   │   │   │   │       │               │           │
 ─── ─── ───         ───         ─── ─── ───                     ─── ─── ─── ─── ───     ─── ───     
│       │   │   │               │       │       │   │       │           │       │ 

         ─── ───     ───     ─── ───     ─── ─── ───         ───             ───     ───     ───     
│           │   │   │                                   │   │   │   │   │   │           │   │   │   │
 ───                 ─── ─── ─── ───         ─── ───     ───             ─── ───     ───             
│       │   │               │       │   │   │       │           │                   │       │       │
     ─── ─── ─── ─── ─── ───         ───                 ───         ─── ─── ───     ─── ───     ─── 
│               │       │   │   │       │   │   │   │   │   │   │   │           │   │               │
         ───         ───             ───     ───     ───                 ─── ───     ───     ───     
│   │       │   │   │           │       │                   │   │   │                   │   │   │   │
 ───     ───             ───     ─── ───     ─── ─── ─── ───         ─── ─── ───     ───             
│       │       │   │   │   │   │           │                   │           │     

     ───     ───     ─── ───         ─── ───         ─── ───     ─── ─── ─── ───     ───     ───     
│       │       │   │                       │               │                                       │
         ───         ─── ───         ───         ─── ───     ───     ───             ───     ─── ─── 
│   │   │               │       │       │       │       │       │   │       │   │   │               │
     ───     ───     ─── ─── ───     ───     ───             ─── ───                 ───     ───     
│           │                       │   │   │       │   │   │       │   │   │   │       │       │   │
     ─── ───     ─── ─── ─── ─── ───     ───             ─── ───     ─── ───             ───     ─── 
│           │   │   │                           │   │           │       │       │   │   │   │       │
                         ─── ───     ───     ───     ─── ───         ───                             
│   │   │   │       │   │   │       │   │   │   │   │   │   │   │           │   │ 

In [262]:
sr = 0
for i in range(100):
    a = wall_to_norm(B2_5(100,100))
    sr += count_no_way(a)
print(sr, sr/100/100)

233430 23.343000000000004


In [263]:
sr = 0
for i in range(100):
    a = wall_to_norm(B2_5(100,100))
    sr += rr_way(a)
print(sr, sr/100/100)

39900 3.99


In [265]:
sr_now_way = 0
sr_rr_way = 0
sr_time_gen = 0
for i in range(100):
    start = time.time()
    a = Kruskal(100,100)
    sr_time_gen = time.time() - start
    sr_now_way += count_no_way(a)
    sr_rr_way += rr_way(a)
print("Kruskal", sr_now_way/100/100, sr_rr_way/100/100, sr_time_gen/100, sep="\n\r\t")

Kruskal
	30.5643
	4.2406
	0.02066166400909424


In [266]:
sr_now_way = 0
sr_rr_way = 0
sr_time_gen = 0
for i in range(100):
    start = time.time()
    a = wall_to_norm(B2_3(100,100))
    sr_time_gen = time.time() - start
    sr_now_way += count_no_way(a)
    sr_rr_way += rr_way(a)
print("B2_3", sr_now_way/100/100, sr_rr_way/100/100, sr_time_gen/100, sep="\n\r\t")

B2_3
	29.4818
	2.8526
	0.0033275341987609864


In [267]:
sr_now_way = 0
sr_rr_way = 0
sr_time_gen = 0
for i in range(100):
    start = time.time()
    a = wall_to_norm(B2_4(100,100))
    sr_time_gen = time.time() - start
    sr_now_way += count_no_way(a)
    sr_rr_way += rr_way(a)
print("B2_4", sr_now_way/100/100, sr_rr_way/100/100, sr_time_gen/100, sep="\n\r\t")

B2_4
	27.6586
	4.4696
	0.007122230529785156


In [268]:
sr_now_way = 0
sr_rr_way = 0
sr_time_gen = 0
for i in range(100):
    start = time.time()
    a = wall_to_norm(B2_5(100,100))
    sr_time_gen = time.time() - start
    sr_now_way += count_no_way(a)
    sr_rr_way += rr_way(a)
print("B2_5", sr_now_way/100/100, sr_rr_way/100/100, sr_time_gen/100, sep="\n\r\t")

B2_5
	23.355300000000003
	4.0106
	0.010440130233764649


In [269]:
tt = np.zeros((20, 4))
for id, i in enumerate(range(5, 101, 5)):
    sr = 0
    for u in range(100):
        start = time.time()
        a = Kruskal(i, i)
        sr += time.time() - start
    tt[id][0] = sr/100
    sr = 0
    for u in range(100):
        start = time.time()
        a = B2_3(i, i)
        sr += time.time() - start
    tt[id][1] = sr/100
    sr = 0
    for u in range(100):
        start = time.time()
        a = B2_4(i, i)
        sr += time.time() - start
    tt[id][2] = sr/100
    sr = 0
    for u in range(100):
        start = time.time()
        a = B2_5(i, i)
        sr += time.time() - start
    tt[id][3] = sr/100
    print(id, tt[id])

0 [0.00200647 0.0004075  0.00052656 0.00035727]
1 [0.01048927 0.00215578 0.00358605 0.00233421]
2 [0.0254096  0.00549298 0.00908872 0.00653589]
3 [0.05012384 0.0104695  0.01776101 0.01386669]
4 [0.07830463 0.01720429 0.03023701 0.02401872]
5 [0.1227458  0.02538001 0.04566336 0.03752789]
6 [0.17396248 0.03564076 0.06917542 0.05935186]
7 [0.2349329  0.04726255 0.09518131 0.07865204]
8 [0.29816861 0.06136801 0.12038172 0.10568061]
9 [0.39274405 0.07561237 0.14969493 0.13467624]
10 [0.46405508 0.09405857 0.18618004 0.16856849]
11 [0.56538508 0.11231584 0.2287247  0.21011962]
12 [0.69242295 0.14262285 0.29527814 0.28084485]
13 [0.80595008 0.1596186  0.35474876 0.32936887]
14 [0.92294487 0.18106449 0.40683912 0.3953761 ]
15 [1.10063177 0.20705042 0.47058127 0.48301752]
16 [1.2403941  0.23622436 0.54820035 0.54092866]
17 [1.40667775 0.26521995 0.62909724 0.63871448]
18 [ 1.58905341  0.30219177  0.72595681 13.24705136]


KeyboardInterrupt: 

In [334]:
def B2_6(n, m):
    a = np.zeros((n+1, m+1, 4), dtype=np.bool)
    b = np.zeros((n+1, m+1), dtype=np.int32)
    b[:,0] = 1
    b[:,-1] = 1
    b[0,:] = 1
    b[-1,:] = 1
    lost = (n-1)*(m-1)
    now = 1
    p = [(-1, 0), (0, 1), (1, 0), (0, -1)]
    def _k(k):
        return (k+2)%4
    while lost:
        now += 1
        x, y = randint(1, n-1), randint(1, m-1)
        while (b[x, y]):
            x, y = randint(1, n-1), randint(1, m-1)
        while True:
            #print(x, y)
            #show_nice(wall_to_norm(a))
            #print(b)
            if b[x, y]!=now:
                b[x, y] = now
                lost -= 1
            good = [k for k in range(4) if b[x+p[k][0], y+p[k][1]] != now]
            if not good:
                x, y = randint(1, n-1), randint(1, m-1)
                while (b[x, y]) != now:
                    x, y = randint(1, n-1), randint(1, m-1)
                continue
            for i in range(len(good)):
                if good[i]%2==((x)+(y))%2:
                    for u in range(1):
                        good.append(good[i])
            k = random.choice(good)
            xx, yy = x+p[k][0], y+p[k][1]
            #print("  ->'",xx,yy,b[xx,yy])
            a[x,y,k] = True
            a[xx,yy,_k(k)] = True
            if b[xx,yy] !=0:
                break
            x, y = xx, yy
    return a

In [335]:
a = B2_6(25, 25)
d = wall_to_norm(a)
show_nice(d)

 ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── ─── 
│   │           │           │               │                           │           │               │
     ─── ───         ─── ─── ─── ─── ───     ───         ───     ───             ───     ─── ───     
│               │                       │       │   │   │   │       │       │       │       │       │
             ─── ───         ───     ───         ───         ───     ───     ─── ───         ─── ─── 
│   │   │           │   │   │               │                   │       │   │           │   │       │
 ─── ───     ─── ───     ─── ─── ─── ─── ───     ─── ───         ─── ─── ─── ─── ───     ───     ─── 
│               │       │                   │   │   │   │   │       │   │       │       │           │
 ───     ───             ─── ─── ───     ───             ───     ───         ─── ───         ───     
│   │       │   │   │           │                   │   │       │               │ 

In [330]:
sr_now_way = 0
sr_rr_way = 0
sr_time_gen = 0
for i in range(20):
    start = time.time()
    a = wall_to_norm(B2_6(100,100))
    sr_time_gen += time.time() - start
    sr_now_way += count_no_way(a)
    sr_rr_way += rr_way(a)
print("B2_6", sr_now_way/20/100, sr_rr_way/20/100, sr_time_gen/20, sep="\n\r\t")

B2_6
	34.568000000000005
	4.409
	0.916896378993988
